Bu bölümde çözmeye çalışacak olan problemimiz bir gerçek hayat problemi. Müşteri terk modellemesi çalışması yapacağız. Bu bölümden sonra uçtan uca bir churn modellemesi (müşteri terk modellemesi - markayı terk etmek) yapacağız. Buna yönelik olarak veri setimizi import ettik.

In [2]:
import findspark
findspark.init("C:\spark")

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf  # Conf = configuration

In [3]:
# Spark bağlantısının yapıldığı bilgiler

spark = SparkSession.builder.master("local").appName("pyspark giris").getOrCreate()

sc = spark.sparkContext # Bağlantı bilgisi

sc

<SparkContext master=local appName=pyspark giris>

In [4]:
# Veri setinin okunması

spark_df = spark.read.csv("churn.csv" , header=True , inferSchema=True)

In [5]:
# Veri seti içerisindeki değişkenlerin ifade edilmesi

spark_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
spark_df.cache

<bound method DataFrame.cache of DataFrame[_c0: int, Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Churn: int]>

- Pandas'ta bildiğimiz bazı fonksiyonlar spark'ta farklı çalışırken bazıları da hiç çalışmayacak. Sprak'taki df yapısı kendine özeldir.
- Örneğin head fonskiyonunda pandas'ta ilk 5 elemanı bize gösterirdi ancak sparrk'ta sadece ilk değeri gösteriyor ve hatırladığımız tablo yapısından ziyade satır şeklinde gösteriyor.

In [7]:
spark_df.head()

Row(_c0=0, Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Churn=1)

# Veri Setine İlk Bakış

In [8]:
spark_df.head()

Row(_c0=0, Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Churn=1)

In [9]:
# Aslında pandas'taki head'in karşılığı sprak'ta "show" fonksiyonumuz 

spark_df.show()

+---+-------------------+----+--------------+---------------+-----+---------+-----+
|_c0|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+-------------------+----+--------------+---------------+-----+---------+-----+
|  0|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|     Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
|  5|   Jessica Williams|48.0|      10356.02|              0| 5.12|      8.0|    1|
|  6|        Eric Butler|44.0|      11331.58|              1| 5.23|     11.0|    1|
|  7|      Zachary Walsh|32.0|       9885.12|              1| 6.92|      9.0|    1|
|  8|        Ashlee Carr|43.0|       14062.6|              1| 5.46|     11.0

In [13]:
# Eğer show fonksiyonunun içine 2 yazarsam sadece ilk 2 satırı getirir

spark_df.show(2)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 2 rows



In [14]:
# Eğer show fonks. truncate=True olarak ayarlanmışsa, gösterilecek olan sütunlardaki değerler belirli bir uzunluktan sonra kesilir ve sonuna "..."" eklenir... 
# ... Bu, özellikle çok uzun metinler içeren sütunları görüntülerken faydalıdır.  truncate=True ile sütunlardaki metinler 20 karakterle sınırlandırılır

spark_df.show(2 , truncate=True)


+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 2 rows



In [ ]:
# Gözlem sayısına erişmek istersek

spark_df.count()

900

In [ ]:
# Değişken ve bu değişkenlerin isimlerine erişmek istersek

spark_df.columns

['_c0',
 'Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Churn']

In [ ]:
# Özet istatistiklere erişmek istersek

spark_df.describe().show()

+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|               _c0|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|              Churn|
+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|               900|          900|              900|              900|               900|              900|               900|                900|
|   mean|             449.5|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|259.95191863111916|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|                 0|   Aaron King|             22.0|          

In [ ]:
# SAdece belirli bir sütunun(değişkenin) istatistiklere erişmek istersek

spark_df.describe("Age").show()

+-------+-----------------+
|summary|              Age|
+-------+-----------------+
|  count|              900|
|   mean|41.81666666666667|
| stddev|6.127560416916251|
|    min|             22.0|
|    max|             65.0|
+-------+-----------------+



In [ ]:
# Veri setinin içerisinden değişken seçmek istersek

spark_df.select("Age" , "Years" , "Names").show()

+----+-----+-------------------+
| Age|Years|              Names|
+----+-----+-------------------+
|42.0| 7.22|   Cameron Williams|
|41.0|  6.5|      Kevin Mueller|
|38.0| 6.67|        Eric Lozano|
|42.0| 6.71|      Phillip White|
|37.0| 5.56|     Cynthia Norton|
|48.0| 5.12|   Jessica Williams|
|44.0| 5.23|        Eric Butler|
|32.0| 6.92|      Zachary Walsh|
|43.0| 5.46|        Ashlee Carr|
|40.0| 7.11|     Jennifer Lynch|
|30.0| 5.22|       Paula Harris|
|45.0| 6.64|     Bruce Phillips|
|45.0| 4.84|       Craig Garner|
|40.0|  5.1|       Nicole Olson|
|41.0|  4.3|     Harold Griffin|
|38.0| 6.81|       James Wright|
|45.0| 7.35|      Doris Wilkins|
|43.0| 8.08|Katherine Carpenter|
|53.0| 6.85|     Lindsay Martin|
|46.0| 5.69|        Kathy Curry|
+----+-----+-------------------+
only showing top 20 rows



In [ ]:
# Veri seti içineki sütunlardan filtreleme yapmak istersek
# Örneğin 40 yaşından büyüklerin sayısı:

spark_df.filter(spark_df.Age > 40).count()

524

In [ ]:
# Sütuna göre gruplayarak hangi değerden kaç tane var onu görmek istersek (Churn bizim bağımlı değişkenimiz)
# 1 -> churn olanlar yani markayı terk edenler
# 0 -> churn olmayanlar yani markayı terk etmeyenler

spark_df.groupBy("Churn").count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|    1|  150|
|    0|  750|
+-----+-----+



In [ ]:
# İleri aggregation işlemleri yapmak istersek
# Örneğin Churn'e göre gruplandırdıktan sonra grupların yaş ortalaması:

spark_df.groupBy("Churn").agg({"Age":"mean"}).show()

+-----+-----------------+
|Churn|         avg(Age)|
+-----+-----------------+
|    1|42.99333333333333|
|    0|41.58133333333333|
+-----+-----------------+



In [ ]:
sc.stop()